In [1]:

# Initialize the relavent data structure
flight = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6']
flight_time = {'AUS,DAL': 50, 'DAL,AUS': 50, 'AUS,HOU': 45, 'HOU,AUS': 45, 'DAL,HOU': 65, 'HOU,DAL': 65}
airport_gate_groundtime = {'AUS': 25, 'DAL': 30, 'HOU': 35}

# Function for Military Time to Minutes since Midnight
def Military2Minutes(Military_Time):
    Military_Time_str = str(Military_Time).zfill(4)
    return 60 * int(Military_Time_str[0:2]) + int(Military_Time_str[2:4])

# Function for Minutes since Midnight to Military Time
def Minutes2Military(Minutes_since_Midnight):
    return str(int(Minutes_since_Midnight / 60)).zfill(2) + str(Minutes_since_Midnight % 60).zfill(2)

# Function for export flight schedule
def ExportSchedule(Schedule_potential, i, flight_number, Dep_port, Arr_port, start, end):
    itinerary = flight_number + ' ' + Dep_port + ' ' + Arr_port + ' ' + Minutes2Military(start) + ' ' + Minutes2Military(end)
    Schedule_potential[i].append(itinerary)

# Function for decide which airport to go and update airport status at the same time
def Airport2Go(Schedule_potential, i, current_time, flight_time, flight_number, airport_gate_groundtime, flight_gate_time, flight_status_table, Gate_avail_time_and_sched_ct):
    Departure_Airport = [value for key, value in flight_gate_time.items() if key == flight_number][0][0]
    fly_ground_time = {}
    Temp_airport_gate_time = {}
    Success_schedule = 0
    Candidate_count = 0
    # Identify the potential airport gate that can be selected to fly
    for key,value in flight_time.items():
        if key[0:3] == Departure_Airport[0:3] and Success_schedule == 0:
            fly_ground_time[key] = flight_time[key] + airport_gate_groundtime[key[4:7]]
            for key2,value2 in Gate_avail_time_and_sched_ct.items():
                if key2[0:3] == key[4:7] and current_time + fly_ground_time[key] >= Gate_avail_time_and_sched_ct[key2][0] and current_time + flight_time[key] <= 1321:
                    Candidate_count = Candidate_count + 1
                    Temp_airport_gate_time[key2] = current_time + fly_ground_time[key] + 1
    # print('Gate status:', Gate_avail_time_and_sched_ct)
    if Candidate_count >= 1:
        if ('HOU1' or 'HOU2' or 'HOU3') in Temp_airport_gate_time.keys():
            Temp_HOU_gate = []
            for key in Temp_airport_gate_time.keys():
                if key.find("HOU") != -1:
                    Temp_HOU_gate.append(key)
            Min_count = min([Gate_avail_time_and_sched_ct[key][1] for key in Temp_HOU_gate])
            Temp = [key for key, value in Gate_avail_time_and_sched_ct.items() if Gate_avail_time_and_sched_ct[key][1] == Min_count and key in Temp_HOU_gate]
        else:
            Min_count = min([Gate_avail_time_and_sched_ct[key][1] for key in Temp_airport_gate_time.keys()])
            Temp = [key for key,value in Gate_avail_time_and_sched_ct.items() if Gate_avail_time_and_sched_ct[key][1] == Min_count]
        Min_time = min([value for key,value in Temp_airport_gate_time.items() if key in Temp])
        # print('Potential airport:',Temp_airport_gate_time)
        key2 = [key for key,value in Temp_airport_gate_time.items() if value == Min_time and key in Temp][0]
        # print('Arrival airport:', key2)
        Gate_avail_time_and_sched_ct[key2][0] = Temp_airport_gate_time[key2]
        Gate_avail_time_and_sched_ct[key2][1] += 1
        flight_status_table[flight_number] = Temp_airport_gate_time[key2]
        flight_gate_time[flight_number] = [key2, Temp_airport_gate_time[key2]]
        Success_schedule = 1
        ExportSchedule(Schedule_potential, i, flight_number, Departure_Airport[0:3], key2[0:3], current_time, Temp_airport_gate_time[key2]-airport_gate_groundtime[key2[0:3]]-1)
        # ExportSchedule(flight_number, Departure_Airport, key2, current_time, Temp_airport_gate_time[key2] - 1)
    if Success_schedule == 0:
        # print(Gate_avail_time_and_sched_ct)
        flight_status_table[flight_number] += 1
        Gate_avail_time_and_sched_ct[Departure_Airport][0] += 1

# Start Main Process

Schedule_potential = []
for i in range(0,100):
    # Initialize the starting status
    if i > 1:
        del Gate_avail_time_and_sched_ct
        del flight_status_table
        del flight_gate_time
        del current_time
    current_time = 360
    flight_status_table = {'T1': 360, 'T2': 360, 'T3': 360, 'T4': 360, 'T5': 360, 'T6': 360}
    flight_gate_time = {'T1':['AUS1',360], 'T2':['DAL1',360], 'T3':['DAL2',360], 'T4':['HOU1',360], 'T5':['HOU2',360], 'T6':['HOU3',360]}
    Gate_avail_time_and_sched_ct = {'AUS1': [360,0], 'DAL1': [360,0], 'DAL2': [360,0], 'HOU1': [360,0], 'HOU2': [360,0], 'HOU3': [360,0]}
    Schedule_potential.append([])
    for current_time in range(360,1256):
        for flight_number in flight:
            if current_time >= flight_status_table[flight_number]:
                Airport2Go(Schedule_potential, i, current_time, flight_time, flight_number, airport_gate_groundtime, flight_gate_time, flight_status_table, Gate_avail_time_and_sched_ct)
    print(Gate_avail_time_and_sched_ct)


{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [1300, 10], 'DAL2': [1300, 10], 'HOU1': [1291, 11], 'HOU2': [1300, 10], 'HOU3': [1300, 10]}
{'AUS1': [1281, 11], 'DAL1': [13